In [1]:
from emulator_utils.pre_process import log_standard, minmax, unscale
from emulator_utils.read_data import readpowerspec
from emulator_utils.split import random_holdout
from emulator_utils.surrogates import simple_mlp, train_mlp, save_mlp, load_mlp

In [2]:
import matplotlib.pylab as plt
import numpy as np
from glob import glob

In [ ]:
scaled, scaler = log_standard(pk_all)

In [ ]:
params = np.arange(scaled.shape[0]).reshape(-1, 1)
scaled_y, scaler_y = minmax(params)

In [ ]:
pk_train, pk_test, param_train, param_test = random_holdout(scaled, params, split_fraction = 0.2)

In [ ]:
mlp = simple_mlp(input_shape=param_train.shape[1], 
                 output_shape=pk_train.shape[1], 
                 hidden_dims=[16, 128, 1024, 4096])

### Setting Hyper-parameters

In [ ]:
learning_rate = 0.01
decay_rate = 0.01
batch_size = 10
num_epochs = 100

In [ ]:
mlp, train_history = train_mlp(mlp, 
                    param_train, pk_train, param_test, pk_test, 
                    learning_rate, decay_rate, num_epochs, batch_size)

In [ ]:
save_mlp(mlp, '../test/model/phot_mlp')
mlp = load_mlp('../test/model/phot_mlp')

In [ ]:
mlp_pred = mlp.predict(param_test)
pk_pred = unscale(mlp_pred, scaler)

### Local explainers

In [ ]:
from emulator_utils.explainer import shap_estimate
from emulator_utils.explainer import plot_shap_summary_single, plot_shap_summary_multiple
from emulator_utils.explainer import plot_shap_force_single, plot_shap_force_multiple

In [ ]:
explainer, shap_values, expected_values = shap_estimate(mlp, param_train, param_test, ['redshift'], ['pk'])

In [ ]:
output_indx = 100  ## k-bin value
plot_shap_summary_single(shap_values[output_indx], param_test, ['redshift'], plot_type='violin')

In [ ]:
plot_shap_summary_multiple(shap_values[:4000:200], param_test, ['redshift'], plot_type='bar')

In [ ]:
plot_shap_force_multiple(expected_values, shap_values, ['redshift'], ['pk'], 0)

In [ ]:
plot_shap_force_single(expected_values, shap_values, ['redshift'], ['pk'], 0, 11)